In [ ]:
!pip install tqdm

In [ ]:
import random
import re
from time import sleep

import pandas as pd
from selenium import webdriver

import tqdm

In [ ]:
def obtener_lista_actores_unicos(ficheros_csv, columna_actor):
    
    # Lista para almacenar todos los DataFrames
    total = []

    # Iterar sobre los archivos y cargarlos en DataFrames
    for fichero in ficheros_csv:
        df = pd.read_csv(fichero, index_col= "actores")
        total.append(df)

    # Concatenar todos los DataFrames en uno solo
    df_concatenado = pd.concat(total)
    
    # Eliminar duplicados
    df_concatenado = df_concatenado.drop_duplicates(subset=[columna_actor])
    print(df_concatenado.head())

    # devolver lista de actores
    return list(df_concatenado[columna_actor])

In [ ]:
def guardar_resultados(lista_datos, fichero_csv):
    df = pd.DataFrame(lista_datos)
    df.columns = ["actor", "año nacimiento", "que hace", "conocido por", "lista premios"]
    df.to_csv(fichero_csv)

In [ ]:
def obtener_informacion_actores(lista_actores, guardar_en_csv):
    
    # cargar chrome en español
    options = webdriver.ChromeOptions()
    options.add_argument("--accept-lang=es")
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()

    # variable para los descansos
    proximo_descanso = random.randint(50,200)
    
    #Definimos una lista para guardar los resultados 
    resultados = []
    
    # Iterar sobre los actores del DataFrame
    for indice, actor in enumerate(tqdm.tqdm(lista_actores)):
        #Hace cada vez que itera 200 actores 
        if indice % 200 == 0 and indice > 0: 
            print(f"Guardando copia parcial de actores en {guardar_en_csv}. Van {indice + 1} actores.")
            guardar_resultados(resultados, guardar_en_csv)
            
        # descanso de vez en cuando
        if indice > proximo_descanso:
            print("Descansando un poco")
            proximo_descanso = indice + random.randint(50,200)
            sleep(random.uniform(10,30))
        try:
            #obtener información del actor de la funcion obtener_informacion_actor 
            info_actor = obtener_informacion_actor(driver, actor)
            resultados.append(info_actor)
        except:
            pass
        
    print(f"Guardando la información de los actores en {guardar_en_csv}")
    guardar_resultados(resultados, guardar_en_csv)
    return resultados


In [ ]:
def obtener_informacion_actor(driver, actor):
    # Abre la página de IMDB para buscar al actor
    driver.get(f"https://www.imdb.com/search/name/?name={actor}")
    sleep(random.uniform(0.8, 1.6))

    # Click en el primer resultado
    driver.find_element("tag name","h3").click()
    sleep(random.uniform(0.8, 1.6))

    # Obtener información sobre el actor
    try:
        que_hace = driver.find_element('xpath',
            '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
        que_hace =  que_hace.split("\n")
    except:
        que_hace = ["Reparto"]
    allweb = driver.find_element("xpath","/html/body").text
    try:
        anio_nacimiento = re.compile(r"\d{1,2} de \w+ de (\d{4})").findall(allweb)[0]
    except:
        anio_nacimiento = None

    # Click en la biografía
    driver.find_element('class name',"ipc-html-content-inner-div").click()
    sleep(random.uniform(0.8, 1.6))

    # Obtener enlaces de películas conocidas
    bio = driver.find_element('xpath', '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/div[2]')
    try:
        enlaces = [x.text for x in bio.find_elements("class name", "ipc-md-link")]
        re_pelicula_anio = r"([\w|\s]+)\s\(\d{4}\)"
        conocido_por = [re.match(re_pelicula_anio, enlace).group(0) for enlace in enlaces if
                        re.match(re_pelicula_anio, enlace)]
    except:
        conocido_por = None

    try:
        # Click en la sección de premios
        driver.find_element('xpath',
            '//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[1]/div/div/div[2]/ul/li[2]/a').click()
        sleep(random.uniform(1.1, 2.1))
    
        # Obtener premios ganados
        premios = driver.find_elements('tag name',"section")[6:]
        lista_premios = [premio.text.split("\n")[0] for premio in premios if "Ganador" in premio.text] 
    except:
        lista_premios = None

    # Guardar información en la lista de resultados
    return (actor, anio_nacimiento, que_hace, conocido_por, lista_premios)

In [ ]:
!dir

In [12]:
# Uso de la función
ficheros =["pelis_concat\\actores_accion_unido.csv"]
columna = "actores" 
fichero_donde_guardar = "informacion_actores.csv"

# lista_actores = obtener_lista_actores_unicos(ficheros, columna)

In [13]:
df_actores = pd.read_csv("pelis_concat\\actores_accion_unido.csv")
lista_actores = df_actores["actores"].to_list()
lista_actores = random.choices(lista_actores, k=random.randint(2500,5000))
# datos_actores contiene los datos de los actores, aunque ya deberían haber sido guardados en el fichero_donde_guardar
datos_actores = obtener_informacion_actores(lista_actores, fichero_donde_guardar)

  2%|▏         | 72/4422 [16:27<14:48:45, 12.26s/it]

Descansando un poco


  5%|▍         | 200/4422 [46:44<17:12:51, 14.68s/it]

Guardando copia parcial de actores en informacion_actores.csv. Van 201 actores.


  5%|▍         | 212/4422 [49:27<14:49:09, 12.67s/it]

...

Guardando copia parcial de actores en informacion_actores.csv. Van 3401 actores.


 77%|███████▋  | 3409/4422 [13:39:53<6:49:33, 24.26s/it]

Descansando un poco


 79%|███████▉  | 3493/4422 [13:40:22<3:38:11, 14.09s/it]


KeyboardInterrupt: 

In [ ]:
#libreria que sirve para coger todos los archivos de la dirección facilitada de manera automática 
import glob

base_dir = "../../../proyecto_2/Adalab-proyecto-da-promo-G-modulo-2-team-1"
# !ls {base_dir}
#Encuentra todos los ficheros que cumplen el siguiente patron
lista_ficheros = glob.glob(f'{base_dir}/**/actores*.csv')
columna = "0"  # o " actores"
fichero_donde_guardar = "informacion_actores.csv"

In [ ]:
lista_actores = obtener_lista_actores_unicos(lista_ficheros, columna)
print(len(lista_actores))
lista_actores

In [ ]:
datos_actores = obtener_informacion_actores(lista_actores[:200], fichero_donde_guardar)

In [ ]:
len(datos_actores)